In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 为在线预测训练图像分类模型的AutoML

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_image_classification_online_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/automl_image_classification_online_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/automl//automl_image_classification_online_prediction.ipynb">
       <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI工作台中打开
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Vertex AI SDK创建图像分类模型，并使用Google Cloud的[AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users)模型进行在线预测。

了解更多关于[从图像分类模型获取预测](https://cloud.google.com/vertex-ai/docs/image-data/classification/get-predictions)的信息。

### 目标

在本教程中，您将使用Vertex SDK从Python脚本创建一个AutoML图像分类模型，并部署用于在线预测。您还可以选择使用`gcloud`命令行工具或在Cloud Console上线创建和部署模型。

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 部署`Model`资源到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

### 数据集

本教程使用的数据集是来自[TensorFlow数据集](https://www.tensorflow.org/datasets/catalog/overview)的[花卉数据集](https://www.tensorflow.org/datasets/catalog/tf_flowers)。在本教程中使用的数据集版本存储在公共云存储桶中。训练模型将预测图像所属的花卉类型，包括五种花卉：雏菊、蒲公英、玫瑰、向日葵或郁金香。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您的预期使用情况生成成本估算。

安装最新版本的Python Vertex AI SDK。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                 tensorflow

### 仅限Colab：取消注释以下单元格以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### 验证您的Google Cloud账户

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明操作。

1. Vertex AI 工作台
* 无需操作，因为您已经通过身份验证。

2. 本地JupyterLab实例，取消注释并运行:

In [ ]:
# ! gcloud auth login

3. 协作，取消注释并运行:

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请参考以下网址了解如何将云存储权限授予您的服务账户：https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples。

### 创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时：运行以下单元格来创建您的云存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import os

import google.cloud.aiplatform as aiplatform

初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化 Python 版本的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# 教程

现在你已经准备好开始创建自己的AutoML图像分类模型。

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

快速查看您的数据

此教程使用存储在公共云存储桶中的一种花卉数据集版本，使用CSV索引文件。

首先快速查看数据。您可以通过计算CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

### 创建数据集

接下来，使用 `ImageDataset` 类的 `create` 方法创建 `Dataset` 资源，该方法需要以下参数：

- `display_name`：`Dataset` 资源的可读名称。
- `gcs_source`：要导入数据项到 `Dataset` 资源中的一个或多个数据集索引文件列表。
- `import_schema_uri`：数据项的数据标记模式。

此操作可能需要几分钟时间。

In [ ]:
dataset = aiplatform.ImageDataset.create(
    display_name="Flowers",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

print(dataset.resource_name)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

#### 创建训练管道

使用`AutoMLImageTrainingJob`类创建一个AutoML训练管道，具有以下参数：

- `display_name`：`TrainingJob`资源的人类可读的名称。
- `prediction_type`：要为其训练模型的任务类型。
  - `classification`：图像分类模型。
  - `object_detection`：图像目标检测模型。
- `multi_label`：如果是分类任务，则是单标签(`False`)还是多标签(`True`)。
- `model_type`：部署模型的类型。
  - `CLOUD`：在Google Cloud上部署。
  - `CLOUD_HIGH_ACCURACY_1`：针对在Google Cloud上部署时准确性优先于延迟进行优化。
  - `CLOUD_LOW_LATENCY_`：针对在Google Cloud上部署时延迟优先于准确性进行优化。
  - `MOBILE_TF_VERSATILE_1`：在边缘设备上部署。
  - `MOBILE_TF_HIGH_ACCURACY_1`：针对在边缘设备上部署时准确性优先于延迟进行优化。
  - `MOBILE_TF_LOW_LATENCY_1`：针对在边缘设备上部署时延迟优先于准确性进行优化。
- `base_model`：（可选）从现有的`Model`资源进行迁移学习--仅支持图像分类。

实例化的对象是训练作业的DAG（有向无环图）。

In [ ]:
dag = aiplatform.AutoMLImageTrainingJob(
    display_name="flowers",
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

print(dag)

#### 运行训练流水线

接下来，你可以通过调用 `run` 方法来运行DAG以启动训练作业，使用以下参数：

- `dataset`: 要训练模型的 `Dataset` 资源。
- `model_display_name`: 已训练模型的人类可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留出数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `budget_milli_node_hours`:（可选）以毫小时为单位指定的最大训练时间（1000 = 小时）。
- `disable_early_stopping`: 如果为 `True`，则在服务认为无法进一步改善模型目标测量之前，训练可能会在使用整个预算之前完成。

完成 `run` 方法后，将返回 `Model` 资源。

执行训练流水线将需要最多20分钟。

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="flowers",
    training_fraction_split=0.8,
    validation_fraction_split=0.1,
    test_fraction_split=0.1,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

## 检查模型评估分数

在模型训练结束后，您可以使用 `list_model_evaluations()` 方法来查看它的评估分数。该方法将为每个评估片段返回一个迭代器。

In [ ]:
model_evaluations = model.list_model_evaluations()

for model_evaluation in model_evaluations:
    print(model_evaluation.to_dict())

部署模型

接下来，部署您的模型进行在线预测。要部署模型，您需要调用“deploy”方法。

In [ ]:
endpoint = model.deploy()

发送一个在线预测请求

向您部署的模型发送一个在线预测。

获取测试项目

您将使用数据集中的任意示例作为测试项目。不必担心该示例可能已在训练模型中使用过 -- 我们只是想演示如何进行预测。

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

### 进行预测

现在您的 `Model` 资源已部署到一个 `Endpoint` 资源中，您可以通过向 Endpoint 资源发送预测请求来进行在线预测。

#### 请求

在本示例中，由于您的测试项目位于 Cloud Storage 存储桶中，您通过使用 `tf.io.gfile.Gfile()` 打开和读取图像的内容。为了将测试数据传递给预测服务，您将字节编码为 base64 -- 这样可以使内容在通过网络传输二进制数据时免受修改。

每个实例的格式如下：

    { 'content': { 'b64': base64编码的字节 } }

由于 `predict()` 方法可以接受多个项（实例），请将您的单个测试项目作为一个测试项目列表发送。

#### 响应

`predict()` 调用的响应是一个 Python 字典，具有以下条目：

- `ids`：每个预测请求的内部分配的唯一标识符。
- `displayNames`：每个类别标签的类名。
- `confidences`：每个类别标签的预测置信度，介于0和1之间。
- `deployed_model_id`：执行预测的部署的 Model 资源的 Vertex AI 标识符。

In [ ]:
import base64

import tensorflow as tf

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances = [{"content": base64.b64encode(content).decode("utf-8")}]

prediction = endpoint.predict(instances=instances)

print(prediction)

## 下线模型

当您完成了预测后，您需要从`Endpoint`资源中下线模型。这将取消所有计算资源分配，并结束对部署模型的计费。

In [ ]:
endpoint.undeploy_all()

清理

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object
dataset.delete()

try:
    endpoint.undeploy_all()
    endpoint.delete()
except Exception as e:
    print(e)

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML trainig job
dag.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI